In [13]:
import sqlite3 as sql3
import pandas as pd
import os

# Caminho para o arquivo CSV
caminho_arquivo = os.path.join('../../database', 'Pokemon.csv')

# Caminho absoluto para o arquivo
nome_csv = os.path.abspath(caminho_arquivo)
nome_banco = os.path.abspath('../../database/Pokemon.db')

# Conectando ao banco e importando o CSV usando pandas
conexao = sql3.connect(nome_banco)
df = pd.read_csv(nome_csv)

In [14]:
# Criando a tabela 'types' no banco de dados
types = pd.DataFrame(df['Type 1'].unique(), columns=['Type 1'])
types.rename(columns={'Type 1': 'type'}, inplace=True)
types.reset_index(inplace=True)
types.rename(columns={'index': 'id'}, inplace=True)

# Exportando a tabela 'types' para o banco de dados
types.to_sql('types', conexao, if_exists='replace', index=False)

18

In [15]:
# Criando a tabela 'generation' no banco de dados
generation = pd.DataFrame(df['Generation'].unique(), columns=['Generation'])
generation.reset_index(inplace=True)
generation.rename(columns={'index': 'id'}, inplace=True)
generation.to_sql('generation', conexao, if_exists='replace', index=False)


6

In [16]:
# Atualizando as gerações no dataframe
consulta = '''SELECT id, Generation FROM generation;'''
ids = pd.read_sql_query(consulta, conexao)
for x in ids['id']:
    y = ids[ids['id'] == x]['Generation'].iloc[0]
    df.loc[df['Generation'] == y, 'Generation'] = x

# Atualizando os tipos no dataframe
consulta = '''SELECT id, type FROM types;'''
ids = pd.read_sql_query(consulta, conexao)

# Recarregando o CSV original para atualizar os tipos
pk = pd.read_csv(nome_csv)
for x in ids['id']:
    y = ids[ids['id'] == x]['type'].iloc[0]
    pk.loc[pk['Type 1'] == y, 'Type 1'] = x
    pk.loc[pk['Type 2'] == y, 'Type 2'] = x

# Substituindo valores NaN em 'Type 1' e 'Type 2' por string vazia
pk.loc[pk['Type 1'].isna(), 'Type 1'] = None
pk.loc[pk['Type 2'].isna(), 'Type 2'] = None

In [17]:
# Remover Pokémon com nome que contém "Mega"
pk1 = pk[~pk['Name'].str.contains('Mega ', case=False, na=False)]
pk1 = pk1[~pk1['Name'].str.contains('Primal', case=False, na=False)]

# Selecionando as colunas que você quer
pk1 = pk1[['Name', 'Type 1', 'Type 2', 'Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Generation', 'Legendary']]

# Resetando o índice e renomeando a coluna de índice
pk1.reset_index(inplace=True)
pk1.rename(columns={'index': 'id','Sp. Def':'SpDef','Sp. Atk':'SpAtk','Type 1':'Type1','Type 2':'Type2'}, inplace=True)

# Atualizando a coluna 'Generation' no DataFrame pk1 com os IDs correspondentes
contagem=0
for index, row in generation.iterrows():
    pk1.loc[pk1['Generation'] == row['Generation'], 'Generation'] = row['id']
    contagem+=1
# Salvando o DataFrame final na tabela 'pokemon'
id_list=[int(x) for x in range(len(pk1))]
pk1['id']=id_list
pk1.to_sql('pokemon', conexao, if_exists='replace', index=False)

750

In [18]:
pk2 = pk[pk['Name'].str.contains('Mega |Primal', case=False, na=False)]
# Removendo as colunas 'Generation' e 'Legendary'
pk2.drop(columns=['Generation', 'Legendary'], inplace=True)
pk2.rename(columns={'#': 'id','Sp. Def':'SpDef','Sp. Atk':'SpAtk','Type 1':'Type1','Type 2':'Type2'}, inplace=True)
# Consulta para pegar os ids dos Pokémon
consulta = '''SELECT id, Name FROM pokemon;'''
ids = pd.read_sql_query(consulta, conexao)

# Inicializando listas para IDs e contagem
original_id = []
new_ids = []
contagem = 0

# Atualizando as Mega Evoluções
for index, row in pk2.iterrows():
    mega_name = row['Name']
    
    # Tentando pegar o nome original, caso seja composto por mais de uma palavra
    split_name = mega_name.split(' ')
    original_name = ' '.join(split_name[1:]) if len(split_name) > 1 else split_name[0]
    
    # Buscando o id do Pokémon original
    try:
        original_pokemon = split_name[1]  # nome do Pokémon original, assume-se que está na segunda posição
        query = f"SELECT id FROM pokemon WHERE Name = '{original_pokemon}'"
        original_id_value = conexao.execute(query).fetchone()[0]  # Pegando o ID do Pokémon original
        original_id.append(original_id_value)
    except (IndexError, TypeError) as e:
        print(f"Pokémon original '{mega_name}' não encontrado no banco de dados.")
        original_id.append(None)  # Adicionando None caso não encontre
        continue  # Pula para a próxima iteração se o Pokémon original não for encontrado

    # Atualizando o nome e ID no DataFrame
    pk2.loc[pk2['Name'] == mega_name, 'Name'] = original_name
    new_ids.append(contagem)  # Adicionando o índice de contagem para associar ao Pokémon Mega
    contagem += 1

# Atribuindo os IDs e original_id às novas colunas
pk2['original'] = original_id
pk2['id'] = new_ids

# Salvando as Mega Evoluções no banco de dados
pk2.to_sql('Mega', conexao, if_exists='replace', index=False)

# Fechando a conexão com o banco de dados
conexao.close()

/tmp/ipykernel_2042/1929537803.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pk2.drop(columns=['Generation', 'Legendary'], inplace=True)
/tmp/ipykernel_2042/1929537803.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pk2.rename(columns={'#': 'id','Sp. Def':'SpDef','Sp. Atk':'SpAtk','Type 1':'Type1','Type 2':'Type2'}, inplace=True)
/tmp/ipykernel_2042/1929537803.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind